## Industry distribution map
<h3>This note book would take an industry name as input, generate a company map of chosen industry in the US.</h3>
<h3>Industry name as listed in Yahoo finance: https://biz.yahoo.com/p/sum_conameu.html </h3>
<h3>Copyright © Kevin Wang 2017. All Rights Reserved.</h3>
<h3>Contac</h3>

In [ ]:
!pip install gmplot
!pip install folium

In [1]:
import requests  
from lxml import html  
import re  
import pandas as pd
from bs4 import BeautifulSoup
import httplib2
from bs4 import SoupStrainer
import folium

In [2]:
def get_location(symbol):
    output=[]
    url='https://www.google.com/finance?q='+str(symbol)
    response = requests.get(url)
    page_data_soup = BeautifulSoup(response.content,'lxml')
    links=[]
    possible_links = page_data_soup.find_all('a')
    for link in possible_links:
        if link.has_attr('href'):
            links.append(link.attrs['href'])
    for i in range(len(links)):
        if links[i][7:10]=='map':
            output.append(links[i])        
    a=output
    address=a[0][a[0].find("&")+3:a[0].find("%")]
    url="https://maps.googleapis.com/maps/api/geocode/json?address=%s" % (address)
    response = requests.get(url)
    json_data = response.json()
    lat=json_data['results'][0]['geometry']['location']['lat']
    lng=json_data['results'][0]['geometry']['location']['lng']
    return [lat,lng]

In [3]:
def company_description(symbol):
    url='https://www.google.com/finance?q='+str(symbol)
    response = requests.get(url)
    try:
        page_data_soup = BeautifulSoup(response.content,'lxml')
        summary=page_data_soup.find('div',{'class':"companySummary"})
        content=summary.text[1:summary.text.rfind(".")]
        return content
    except:
        return "Can not find this company description!"

In [4]:
def sector_symbols(sector_name):
    http = httplib2.Http()
    status,response = http.request('https://biz.yahoo.com/p/sum_conameu.html')
    links=[]
    for link in BeautifulSoup(response, parseOnlyThese=SoupStrainer(['a','href'])):
        links.append(link)
    link_list=[]
    for link in links:
        a=link
        b=str(a)
        c=b[b.find("href=")+6:b.find("href=")+21]
        if 48<=ord(c[0])<=57:
            link_list.append(b[b.find("href=")+6:b.find("href=")+21])

    table_secotrs=pd.read_html("https://biz.yahoo.com/p/sum_conameu.html")
    sectors_name=table_secotrs[2][0][2:]
    sectors_name.index=range(len(sectors_name))

    for k in range(len(sectors_name)):
        if sectors_name[k]==sector_name:
            link="https://biz.yahoo.com/p/"+link_list[k]
            tables=pd.read_html(link)
            symbol_list=[]
            for thing in tables[2][0][5:]:
                symbol=thing[thing.find("(")+1:thing.find(")")]
                left=thing[thing.find(")")+1:]
                if left.find("(") == -1:
                    if symbol.find(".")<0:
                        symbol_list.append(symbol)
                else:
                    symbol=left[left.find("(")+1:left.find(")")]
                    if symbol.find(".")<0:
                        symbol_list.append(symbol)
    return symbol_list

In [7]:
def map(sectorname):  
    symbols=sector_symbols(sectorname)
    latitudes=[]
    longitudes=[]
    symbol_show=[]
    for thing in symbols:
        try:
            per=get_location(thing)
            latitudes.append(per[0])
            longitudes.append(per[1])
            symbol_show.append(company_description(thing))
        except:
            continue
    map_1 = folium.Map(location=[40, -99], zoom_start=4)
    map_2 = folium.Map(location=[40, -99], tiles='Stamen Terrain',zoom_start=4)
    for i in range(len(symbol_show)):
        folium.Marker([latitudes[i], longitudes[i]], popup=symbol_show[i]).add_to(map_1)
    map_1.save('Health Care Plans.html')
    print("Map drawed! Named sector.html. Please look your file")

In [9]:
map('Health Care Plans')

/Users/yaolinwang/anaconda/lib/python3.5/site-packages/bs4/__init__.py:133: UserWarning: The "parseOnlyThese" argument to the BeautifulSoup constructor has been renamed to "parse_only."
  'has been renamed to "%s."' % (old_name, new_name))
/Users/yaolinwang/anaconda/lib/python3.5/site-packages/bs4/__init__.py:181: UserWarning: No parser was explicitly specified, so I'm using the best available HTML parser for this system ("lxml"). This usually isn't a problem, but if you run this code on another system, or in a different virtual environment, it may use a different parser and behave differently.

The code that caused this warning is on line 184 of the file /Users/yaolinwang/anaconda/lib/python3.5/runpy.py. To get rid of this warning, change code that looks like this:

 BeautifulSoup([your markup])

to this:

 BeautifulSoup([your markup], "lxml")

  markup_type=markup_type))


Map drawed! Named sector.html. Please look your file


In [ ]:
industry sample:
    
Diagnostic Substances
Diversified Electronics
Health Care Plans